# 数理最適化とは

# 線形最適化問題

In [3]:
from gurobipy import *
model = Model("lo1")
x1 = model.addVar(name="x1")
x2 = model.addVar(name="x2")
x3 = model.addVar(ub=30, name="x3")
model.update()
model.addConstr(2*x1 + x2 + x3 <= 60)
model.addConstr(x1 + 2*x2 + x3 <= 60)
model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)
model.optimize()
print("Opt. Value", model.ObjVal)
for v in model.getVars():
    print(v.VarName, v.X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x2b324efa
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [3e+01, 3e+01]
  RHS range        [6e+01, 6e+01]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+31   3.000000e+30   3.300000e+01      0s
       3    1.2300000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  1.230000000e+03
Opt. Value 1230.0
x1 10.0
x2 10.0
x3 30.0


# 整数最適化問題

In [8]:
from gurobipy import *
model = Model("puzzle")
x = model.addVar(vtype="I")
y = model.addVar(vtype="I")
z = model.addVar(vtype="I")
model.update()
model.addConstr(x + y + z == 32) 
model.addConstr(2*x + 4*y + 8*z ==80)
model.setObjective(y + z, GRB.MINIMIZE)
model.optimize()
print("Opt. Val.", model.ObjVal)
print("(x,y,z)=", x.X, y.X, z.X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x9688a20c
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 8e+01]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
Opt. Val. 4.0
(x,y,z)= 28.0 2.0 2.0


# 輸送問題

In [23]:
# 問題例のデータ準備
d = {1:80, 2:270, 3:250, 4:160, 5:180}
M = {1:500, 2:500, 3:500}
I = [1, 2, 3, 4, 5]
J = [1, 2, 3]
c = {(1,1):4, (1,2):6, (1,3):9,
     (2,1):5, (2,2):4, (2,3):7,
     (3,1):6, (3,2):3, (3,3):4,
     (4,1):8, (4,2):5, (4,3):3,
     (5,1):10, (5,2):8, (5,3):4
    }

In [24]:
model = Model("transportation")
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype="C", name="x({},{})".format(i,j))
model.update()

In [25]:
for i in I:
    model.addConstr(quicksum(x[i,j] for j in J)==d[i], name="Demand{}".format(i))

In [26]:
for j in J:
    model.addConstr(quicksum(x[i,j] for i in I) <= M[j], name="Capacity{}".format(j))

In [27]:
model.setObjective(quicksum(c[i,j]*x[i,j] for(i,j) in x), GRB.MINIMIZE)

In [28]:
model.optimize()
print("Opt. Val", model.ObjVal)
EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity {} from factory {} to customer {}".format(x[i,j].X, j, i))

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xa225ad1d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 5e+02]
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3500000e+03   2.000000e+01   0.000000e+00      0s
       1    3.3700000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  3.370000000e+03
Opt. Val 3370.0
sending quantity 80.0 from factory 1 to customer 1
sending quantity 20.0 from factory 1 to customer 2
sending quantity 250.0 from factory 2 to customer 2
sending quantity 250.0 from factory 2 to customer 3
sending quantity 160.0 from factory 3 to customer 4
sending quant

# 双対問題

In [30]:
print("Const. Name: Slack, Dual")
for c in model.getConstrs():
    print("{}: {}, {}".format(c.ConstrName, c.slack, c.Pi))

Const. Name: Slack, Dual
Demand1: 0.0, 4.0
Demand2: 0.0, 5.0
Demand3: 0.0, 4.0
Demand4: 0.0, 3.0
Demand5: 0.0, 4.0
Capacity1: 400.0, 0.0
Capacity2: 0.0, -1.0
Capacity3: 160.0, 0.0
